In [1]:
import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect, Float
import joblib

In [2]:
engine = create_engine("sqlite:///credit_customer.db")

meta = MetaData()

customer= Table(
    'customer', meta, 
    Column('ID', String, primary_key=True), 
    Column('Customer_ID', String), 
    Column('Name', String),
    Column('Age', Integer), 
    Column('SSN', String), 
    Column('Occupation', String), 
    Column('Annual_Income', Float),
    Column('Monthly_Inhand_Salary', Float)        
)

In [3]:
credit= Table(
    'credit', meta, 
    Column('ID_Credit', Integer, primary_key=True),
    Column('ID', String, ForeignKey("customer.ID")), 
    Column('Num_Bank_Accounts', Integer),
    Column('Interest_Rate', Integer),
    Column('Num_of_Loan', Integer), 
    Column('Type_of_Loan', String),
    Column('Delay_from_due_date', Integer), 
    Column('Num_of_Delayed_Payment', Integer), 
    Column('Changed_Credit_Limit', Float),
    Column('Num_Credit_Inquiries', Float),
    Column('Credit_Mix', String),
    Column('Outstanding_Debt', Float),
    Column('Credit_Utilization_Ratio', Float), 
    Column('Payment_of_Min_Amount', String),
    Column('Total_EMI_per_month', Float),
    Column('Amount_invested_monthly', Float), 
    Column('Payment_Behaviour', String),
    Column('Monthly_Balance', Float),
    Column('Credit_History_Age_Years', Integer),
    Column('Credit_History_Age_Months', Integer)    
)

In [4]:
meta.create_all(engine)

In [5]:
df_customer = joblib.load('./df_customer.pkl')#file with clean data
values = list(zip(*map(df_customer.get, df_customer)))

In [6]:
with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            # We indicate the format of a tuple of this table
            markers = ','.join('?' * len(values[0])) 
            # We use the SQL language in text format where markers is the format of a tuple
            ins = 'INSERT INTO {tablename} VALUES ({markers})'
            # This particular format is specified using the format member function
            ins = ins.format(tablename=customer.name, markers=markers)
            # Finally we can use the tuples created by executing the SQL command
            connection.execute(ins, values)
        except:
            transaction.rollback()
            raise
        else:
            transaction.commit()

In [7]:
df_credit = joblib.load('./df_credit.pkl')#file with clean data
values_credit = list(zip(*map(df_credit.get, df_credit)))

In [8]:
with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            # We indicate the format of a tuple of this table
            markers = ','.join('?' * len(values_credit[0])) 
            # We use the SQL language in text format where markers is the format of a tuple
            ins = 'INSERT INTO {tablename} VALUES ({markers})'
            # This particular format is specified using the format member function
            ins = ins.format(tablename=credit.name, markers=markers)
            # Finally we can use the tuples created by executing the SQL command
            connection.execute(ins, values_credit)
        except:
            transaction.rollback()
            raise
        else:
            transaction.commit()

In [9]:
with engine.connect() as connection:
    results = connection.execute("SELECT * FROM customer LIMIT 1;")
    print(results.fetchall())

with engine.connect() as connection:
    results = connection.execute("SELECT * FROM credit LIMIT 1;")
    print(results.fetchall())

[('0x160a', 'CUS_0xd40', 'Aaron Maashoh', 23, '821-00-0265', 'Scientist', 19114.12, 1824.8433333333328)]
[(0, '0x160a', 3, 3, 19114, 'Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan', 3, 7, 11.27, 2022.0, 'Good', 809.98, 35.03040185583525, 'No', 49.57494921489417, 236.64268203272135, 'Low_spent_Small_value_payments', 186.26670208571772, 22, 9)]


In [10]:
#sql = text('DROP TABLE IF EXISTS customer;')
#result = engine.execute(sql)

In [11]:
#sql = text('DROP TABLE IF EXISTS credit;')
#result = engine.execute(sql)

In [12]:
conn = engine.connect() #Instantiate the Connection class in a variable named conn

In [13]:
#joblib.dump(conn, './conn.pkl')

#PicklingError: Can't pickle <function create_engine.<locals>.connect at 0x000001BB14874A60>: it's not found as sqlalchemy.engine.create.create_engine.<locals>.connect

In [20]:
#Find custumers with max Annual_Income and show there occupation
stmt = text ( "SELECT Occupation,Max(Annual_Income) as MaxIncome FROM customer "
             " GROUP BY Occupation ORDER BY MaxIncome DESC LIMIT 10" )
result = conn.execute(stmt)
result.fetchall()

[('Scientist', 24137255.0),
 ('Mechanic', 24017307.0),
 ('Lawyer', 24004088.0),
 ('Musician', 23960895.0),
 ('Engineer', 23941224.0),
 ('Architect', 23875413.0),
 ('Media_Manager', 23751475.0),
 ('Entrepreneur', 23711825.0),
 ('Journalist', 23572260.0),
 ('Developer', 23334223.0)]

In [53]:
#Find custumers with min Annual_Income and show there occupation
stmt = text ( "SELECT Occupation,MIN(Annual_Income) as MinIncome FROM customer "
             " GROUP BY Occupation ORDER BY MinIncome DESC LIMIT 10" )
result = conn.execute(stmt)
result.fetchall()

[('Journalist', 7171.26),
 ('Mechanic', 7169.585),
 ('Architect', 7103.035),
 ('Manager', 7087.38),
 ('Lawyer', 7084.365),
 ('Teacher', 7079.32),
 ('Entrepreneur', 7077.87),
 ('Developer', 7059.455),
 ('Scientist', 7046.5),
 ('Accountant', 7021.91)]

In [22]:
#Find custumers with average Annual_Income and show there occupation
stmt = text ( "SELECT Occupation,AVG(Annual_Income) as AvgIncome FROM customer "
             " GROUP BY Occupation ORDER BY AvgIncome DESC LIMIT 10" )
result = conn.execute(stmt)
result.fetchall()

[('Lawyer', 209794.51680994648),
 ('Scientist', 209158.45529821832),
 ('Journalist', 185826.77735661314),
 ('Mechanic', 176170.47286391206),
 ('Doctor', 173961.91018612927),
 ('Architect', 173063.70524696016),
 ('Musician', 168130.9838434677),
 ('Media_Manager', 165807.89449139545),
 ('Manager', 161830.06886201468),
 ('Accountant', 161360.28372716866)]

In [24]:
#Find custumers age with max Monthly_Inhand_Salary
stmt = text ( "SELECT Age,Occupation, Max(Monthly_Inhand_Salary) as MaxIncome FROM customer "
             " GROUP BY Age, Occupation ORDER BY MaxIncome DESC LIMIT 10" )
result = conn.execute(stmt)
result.fetchall()

[(44, 'Lawyer', 15204.633333333331),
 (45, 'Lawyer', 15204.633333333331),
 (47, 'Architect', 15167.18),
 (38, 'Architect', 15136.696666666665),
 (48, 'Media_Manager', 15115.19),
 (28, 'Engineer', 15101.94),
 (19, 'Accountant', 15090.076666666668),
 (25, 'Musician', 14978.336666666668),
 (19, 'Writer', 14960.25),
 (26, 'Mechanic', 14929.54)]

In [25]:
#Find custumers age with average Monthly_Inhand_Salary
stmt = text ( "SELECT Age,Occupation, AVG(Monthly_Inhand_Salary) as AVGIncome FROM customer "
             " GROUP BY Age, Occupation ORDER BY AVGIncome DESC LIMIT 10" )
result = conn.execute(stmt)
result.fetchall()

[(50, 'Teacher', 7484.357708333333),
 (47, 'Architect', 7174.1701096491215),
 (54, 'Engineer', 7127.029916666668),
 (50, 'Journalist', 6863.971133826655),
 (50, 'Accountant', 6783.142699411936),
 (51, 'Lawyer', 6686.5755579717825),
 (50, 'Entrepreneur', 6580.346858258982),
 (48, 'Mechanic', 6464.175146127045),
 (49, 'Accountant', 6379.054512076755),
 (47, 'Developer', 6375.531020674111)]

In [54]:
#Find custumers age with MIN Monthly_Inhand_Salary
stmt = text ( "SELECT Age,Occupation, Min(Monthly_Inhand_Salary) as MinIncome FROM customer "
             " GROUP BY Age, Occupation ORDER BY MinIncome DESC LIMIT 10" )
result = conn.execute(stmt)
result.fetchall()

[(95, 'Entrepreneur', 5422.231059336125),
 (14, 'Lawyer', 3085.853333333333),
 (54, 'Musician', 3085.853333333333),
 (95, 'Writer', 3085.853333333333),
 (47, 'Architect', 3010.778333333333),
 (56, 'Engineer', 2968.570833333333),
 (49, 'Writer', 2819.603374504596),
 (56, 'Accountant', 2802.5939186025025),
 (51, 'Scientist', 2502.7133333333336),
 (48, 'Journalist', 2498.629093019256)]

In [27]:
#Show Names which begins with a Pa and which occupation is Scientest
stmt = text ( "SELECT Name, Occupation FROM customer"
             " WHERE Name LIKE 'Pa%' AND Occupation LIKE '%Scientist%' LIMIT 10")
result = conn.execute(stmt)
result.fetchall()

[('Paul Taylorm', 'Scientist'),
 ('Paul Taylorm', 'Scientist'),
 ('Patrick Ruckerw', 'Scientist'),
 ('Patrick Ruckerw', 'Scientist'),
 ('Patrick Ruckerw', 'Scientist'),
 ('Paritosh Bansalm', 'Scientist'),
 ('Paritosh Bansalm', 'Scientist'),
 ('Paritosh Bansalm', 'Scientist'),
 ('Parisa Hafeziu', 'Scientist'),
 ('Palmerc', 'Scientist')]

In [52]:
#Find most frequent value in Name column( from the second value to avoid Nan)!!
#fOUND OUT HOW TO SHOW FROM THE SECOND VALUE

stmt = text ( "SELECT Name, COUNT(Name) AS value_occurrence FROM customer"
             " GROUP BY Name ORDER BY value_occurrence DESC LIMIT 3;")

result = conn.execute(stmt)
result.fetchall()


#MrX=Nan

[('MrX', 3948), ('Stevex', 18), ('Langep', 18)]

In [61]:
#Find average amount invested monthly by musiciens
stmt = text ( "SELECT Occupation, AVG(Amount_invested_monthly) FROM (customer INNER JOIN credit ON customer.ID = credit.ID )"
              "WHERE Occupation LIKE '%Musician%' LIMIT 10" )
result = conn.execute(stmt)
result.fetchall()

[('Musician', 1086.1156203651738)]

In [63]:
#Find average amount invested monthly by Lawyer
stmt = text ( "SELECT Occupation, AVG(Amount_invested_monthly) FROM (customer INNER JOIN credit ON customer.ID = credit.ID )"
              "WHERE Occupation LIKE '%Lawyer%' LIMIT 10" )
result = conn.execute(stmt)
result.fetchall()

[('Lawyer', 1110.5319843204177)]

In [67]:
#Find 
stmt = text ( "SELECT Occupation, Annual_Income, Payment_Behaviour FROM (customer INNER JOIN credit ON customer.ID = credit.ID )"
              "WHERE Annual_Income > 500000" 
             "GROUP BY Occupation ORDER BY Annual_Income DESC" )
result = conn.execute(stmt)
result.fetchall()

[('Engineer', 23824706.0, 'High_spent_Medium_value_payments'),
 ('Lawyer', 20976455.0, 'High_spent_Medium_value_payments'),
 ('Media_Manager', 20601508.0, 'Low_spent_Small_value_payments'),
 ('Musician', 15717110.0, 'High_spent_Small_value_payments'),
 ('Developer', 13740470.0, 'High_spent_Small_value_payments'),
 ('Journalist', 11521978.0, 'Low_spent_Medium_value_payments'),
 ('Accountant', 11092049.0, 'Low_spent_Medium_value_payments'),
 ('Scientist', 10527110.0, 'High_spent_Small_value_payments'),
 ('Manager', 10125936.0, 'High_spent_Medium_value_payments'),
 ('Architect', 7873905.0, 'High_spent_Medium_value_payments'),
 ('Teacher', 4602545.0, 'High_spent_Large_value_payments'),
 ('Entrepreneur', 4148862.0, 'Low_spent_Small_value_payments'),
 ('Mechanic', 3492542.0, 'High_spent_Medium_value_payments'),
 ('Doctor', 3035017.0, 'High_spent_Medium_value_payments'),
 ('Writer', 671709.0, 'Low_spent_Small_value_payments')]

In [70]:
#Show all occupations
stmt = text ( "SELECT Occupation, COUNT(*) FROM customer GROUP BY Occupation" )
result = conn.execute(stmt)
result.fetchall()

[('Accountant', 2628),
 ('Architect', 2632),
 ('Developer', 2635),
 ('Doctor', 2552),
 ('Engineer', 2676),
 ('Entrepreneur', 2600),
 ('Journalist', 2563),
 ('Lawyer', 2815),
 ('Manager', 2522),
 ('Mechanic', 2638),
 ('Media_Manager', 2615),
 ('Musician', 2434),
 ('Scientist', 2582),
 ('Teacher', 2603),
 ('Writer', 2476)]

In [79]:
#Show

stmt = text ( "SELECT Occupation, Credit_Mix,Annual_Income, Payment_Behaviour FROM (customer INNER JOIN credit ON customer.ID = credit.ID )"
              "WHERE Credit_Mix  LIKE '%Bad%' ORDER BY Annual_Income DESC LIMIT 10")
             
result = conn.execute(stmt)
result.fetchall()

[('Scientist', 'Bad', 23994243.0, 'Low_spent_Small_value_payments'),
 ('Scientist', 'Bad', 23967713.0, 'Low_spent_Small_value_payments'),
 ('Architect', 'Bad', 23875413.0, 'Low_spent_Small_value_payments'),
 ('Engineer', 'Bad', 23824706.0, 'High_spent_Medium_value_payments'),
 ('Musician', 'Bad', 23421410.0, 'Low_spent_Small_value_payments'),
 ('Engineer', 'Bad', 23420380.0, 'High_spent_Small_value_payments'),
 ('Lawyer', 'Bad', 23324584.0, 'High_spent_Medium_value_payments'),
 ('Lawyer', 'Bad', 23213897.0, 'Low_spent_Large_value_payments'),
 ('Manager', 'Bad', 23022922.0, 'High_spent_Medium_value_payments'),
 ('Entrepreneur', 'Bad', 23005365.0, 'Low_spent_Medium_value_payments')]

In [80]:
stmt = text ( "SELECT Occupation, Credit_Mix,Annual_Income, Payment_Behaviour FROM (customer INNER JOIN credit ON customer.ID = credit.ID )"
              "WHERE Credit_Mix  LIKE '%Good%' ORDER BY Annual_Income DESC LIMIT 10")
             
result = conn.execute(stmt)
result.fetchall()

[('Scientist', 'Good', 24112304.0, 'High_spent_Large_value_payments'),
 ('Media_Manager', 'Good', 23751475.0, 'Low_spent_Small_value_payments'),
 ('Developer', 'Good', 23334223.0, 'Low_spent_Small_value_payments'),
 ('Journalist', 'Good', 22705386.0, 'Low_spent_Small_value_payments'),
 ('Developer', 'Good', 22444909.0, 'High_spent_Large_value_payments'),
 ('Developer', 'Good', 21631150.0, 'High_spent_Medium_value_payments'),
 ('Accountant', 'Good', 21485760.0, 'Low_spent_Large_value_payments'),
 ('Scientist', 'Good', 20696625.0, 'High_spent_Large_value_payments'),
 ('Developer', 'Good', 20103710.0, 'Low_spent_Medium_value_payments'),
 ('Lawyer', 'Good', 20055893.0, 'High_spent_Medium_value_payments')]

In [81]:
stmt = text ( "SELECT Age, Occupation, Type_of_Loan FROM (customer INNER JOIN credit ON customer.ID = credit.ID )"
              "WHERE Age = 18 LIMIT 10" )
result = conn.execute(stmt)
result.fetchall()

[(18, 'Musician', 'Home Equity Loan, Credit-Builder Loan, Debt Consolidation Loan, Payday Loan, and Home Equity Loan'),
 (18, 'Musician', 'Home Equity Loan, Credit-Builder Loan, Debt Consolidation Loan, Payday Loan, and Home Equity Loan'),
 (18, 'Musician', 'Home Equity Loan, Credit-Builder Loan, Debt Consolidation Loan, Payday Loan, and Home Equity Loan'),
 (18, 'Media_Manager', 'Home Equity Loan, Mortgage Loan, Debt Consolidation Loan, Home Equity Loan, Personal Loan, and Personal Loan'),
 (18, 'Scientist', 'Payday Loan, and Not Specified'),
 (18, 'Scientist', 'Payday Loan, and Not Specified'),
 (18, 'Scientist', 'Payday Loan, and Not Specified'),
 (18, 'Manager', 'Mortgage Loan, Not Specified, Credit-Builder Loan, Debt Consolidation Loan, Not Specified, Mortgage Loan, and Student Loan'),
 (18, 'Manager', 'Mortgage Loan, Not Specified, Credit-Builder Loan, Debt Consolidation Loan, Not Specified, Mortgage Loan, and Student Loan'),
 (18, 'Manager', 'Mortgage Loan, Not Specified, Credit

In [87]:
stmt = text ( "SELECT Age, Occupation, Type_of_Loan FROM (customer INNER JOIN credit ON customer.ID = credit.ID )"
              "WHERE Age = 40 AND Occupation='Developer'  LIMIT 20" )
result = conn.execute(stmt)
result.fetchall()

[(40, 'Developer', 'Home Equity Loan, Student Loan, and Debt Consolidation Loan'),
 (40, 'Developer', 'Home Equity Loan, Student Loan, and Debt Consolidation Loan'),
 (40, 'Developer', 'Home Equity Loan, Student Loan, and Debt Consolidation Loan'),
 (40, 'Developer', 'Home Equity Loan, Student Loan, and Debt Consolidation Loan'),
 (40, 'Developer', 'Home Equity Loan, Mortgage Loan, Mortgage Loan, Not Specified, and Student Loan'),
 (40, 'Developer', 'Home Equity Loan, Mortgage Loan, Mortgage Loan, Not Specified, and Student Loan'),
 (40, 'Developer', 'Home Equity Loan, Mortgage Loan, Mortgage Loan, Not Specified, and Student Loan'),
 (40, 'Developer', 'Home Equity Loan, Payday Loan, Credit-Builder Loan, Personal Loan, Debt Consolidation Loan, Home Equity Loan, Debt Consolidation Loan, and Student Loan'),
 (40, 'Developer', 'Home Equity Loan, Payday Loan, Credit-Builder Loan, Personal Loan, Debt Consolidation Loan, Home Equity Loan, Debt Consolidation Loan, and Student Loan'),
 (40, 'De

In [90]:
stmt = text ( "SELECT Age, Occupation, Type_of_Loan FROM (customer INNER JOIN credit ON customer.ID = credit.ID )"
              "WHERE Age = 40 AND Occupation='Developer'  LIMIT 10" )
result = conn.execute(stmt)
result.fetchall()

[(40, 'Developer', 'Home Equity Loan, Student Loan, and Debt Consolidation Loan'),
 (40, 'Developer', 'Home Equity Loan, Student Loan, and Debt Consolidation Loan'),
 (40, 'Developer', 'Home Equity Loan, Student Loan, and Debt Consolidation Loan'),
 (40, 'Developer', 'Home Equity Loan, Student Loan, and Debt Consolidation Loan'),
 (40, 'Developer', 'Home Equity Loan, Mortgage Loan, Mortgage Loan, Not Specified, and Student Loan'),
 (40, 'Developer', 'Home Equity Loan, Mortgage Loan, Mortgage Loan, Not Specified, and Student Loan'),
 (40, 'Developer', 'Home Equity Loan, Mortgage Loan, Mortgage Loan, Not Specified, and Student Loan'),
 (40, 'Developer', 'Home Equity Loan, Payday Loan, Credit-Builder Loan, Personal Loan, Debt Consolidation Loan, Home Equity Loan, Debt Consolidation Loan, and Student Loan'),
 (40, 'Developer', 'Home Equity Loan, Payday Loan, Credit-Builder Loan, Personal Loan, Debt Consolidation Loan, Home Equity Loan, Debt Consolidation Loan, and Student Loan'),
 (40, 'De

In [91]:
#Home

stmt = text ( "SELECT Age, Occupation, Type_of_Loan FROM (customer INNER JOIN credit ON customer.ID = credit.ID )"
              "WHERE Age = 18 AND Type_of_Loan LIKE '%Home%'  LIMIT 10" )
result = conn.execute(stmt)
result.fetchall()

[(18, 'Musician', 'Home Equity Loan, Credit-Builder Loan, Debt Consolidation Loan, Payday Loan, and Home Equity Loan'),
 (18, 'Musician', 'Home Equity Loan, Credit-Builder Loan, Debt Consolidation Loan, Payday Loan, and Home Equity Loan'),
 (18, 'Musician', 'Home Equity Loan, Credit-Builder Loan, Debt Consolidation Loan, Payday Loan, and Home Equity Loan'),
 (18, 'Media_Manager', 'Home Equity Loan, Mortgage Loan, Debt Consolidation Loan, Home Equity Loan, Personal Loan, and Personal Loan'),
 (18, 'Developer', 'Home Equity Loan, Credit-Builder Loan, Student Loan, and Personal Loan'),
 (18, 'Developer', 'Home Equity Loan, Credit-Builder Loan, Student Loan, and Personal Loan'),
 (18, 'Entrepreneur', 'Debt Consolidation Loan, Home Equity Loan, and Not Specified'),
 (18, 'Entrepreneur', 'Debt Consolidation Loan, Home Equity Loan, and Not Specified'),
 (18, 'Entrepreneur', 'Debt Consolidation Loan, Home Equity Loan, and Not Specified'),
 (18, 'Engineer', 'Personal Loan, Credit-Builder Loan, 